## Deliverable 2. Create a Customer Travel Destinations Map.

In [70]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [71]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Vostok,46.4856,135.8833,78.94,63,18,1.57,RU,few clouds
1,1,Constitucion,-35.3333,-72.4167,53.96,68,17,7.54,CL,few clouds
2,2,Kendari,-3.9450,122.4989,74.50,93,98,2.39,ID,moderate rain
3,3,Chokurdakh,70.6333,147.9167,57.36,50,100,5.73,RU,overcast clouds
4,4,Laguna,38.4210,-121.4238,104.45,20,1,1.99,US,clear sky


In [72]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [73]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Vostok,46.4856,135.8833,78.94,63,18,1.57,RU,few clouds
2,2,Kendari,-3.9450,122.4989,74.50,93,98,2.39,ID,moderate rain
5,5,Ribeira Grande,38.5167,-28.7000,70.38,85,100,20.42,PT,overcast clouds
6,6,Puerto Ayora,-0.7393,-90.3518,73.54,90,88,3.00,EC,overcast clouds
8,8,Tanabe,33.7333,135.3667,81.61,75,20,1.01,JP,few clouds
9,9,Kapaa,22.0752,-159.3190,86.32,74,90,5.01,US,overcast clouds
10,10,Vaini,-21.2000,-175.2000,78.96,89,20,6.91,TO,few clouds
11,11,Guerrero Negro,27.9769,-114.0611,80.35,55,20,17.98,MX,few clouds
12,12,Alofi,-19.0595,-169.9187,80.49,74,40,10.36,NU,scattered clouds
13,13,Hilo,19.7297,-155.0900,80.80,73,90,6.91,US,light rain


In [74]:
# 4a. Determine if there are any empty rows.
len(preferred_cities_df)

331

In [75]:
np.where(pd.isnull(preferred_cities_df))

(array([], dtype=int64), array([], dtype=int64))

In [76]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_prefer_city_df = preferred_cities_df.dropna()
new_prefer_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Vostok,46.4856,135.8833,78.94,63,18,1.57,RU,few clouds
2,2,Kendari,-3.9450,122.4989,74.50,93,98,2.39,ID,moderate rain
5,5,Ribeira Grande,38.5167,-28.7000,70.38,85,100,20.42,PT,overcast clouds
6,6,Puerto Ayora,-0.7393,-90.3518,73.54,90,88,3.00,EC,overcast clouds
8,8,Tanabe,33.7333,135.3667,81.61,75,20,1.01,JP,few clouds


In [77]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_prefer_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vostok,RU,78.94,few clouds,46.4856,135.8833,
2,Kendari,ID,74.50,moderate rain,-3.9450,122.4989,
5,Ribeira Grande,PT,70.38,overcast clouds,38.5167,-28.7000,
6,Puerto Ayora,EC,73.54,overcast clouds,-0.7393,-90.3518,
8,Tanabe,JP,81.61,few clouds,33.7333,135.3667,
9,Kapaa,US,86.32,overcast clouds,22.0752,-159.3190,
10,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
11,Guerrero Negro,MX,80.35,few clouds,27.9769,-114.0611,
12,Alofi,NU,80.49,scattered clouds,-19.0595,-169.9187,
13,Hilo,US,80.80,light rain,19.7297,-155.0900,


In [78]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # USED _ hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    #IndexError
    #Recall that when trying to parse the data from an API request, we need to use a try-except block to handle the error with a statement and continue the retrieval.
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [79]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vostok,RU,78.94,few clouds,46.4856,135.8833,
2,Kendari,ID,74.50,moderate rain,-3.9450,122.4989,Hotel Wixel
5,Ribeira Grande,PT,70.38,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
6,Puerto Ayora,EC,73.54,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Tanabe,JP,81.61,few clouds,33.7333,135.3667,美吉屋旅館


In [80]:
np.where(pd.isnull(hotel_df))

(array([], dtype=int64), array([], dtype=int64))

In [81]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vostok,RU,78.94,few clouds,46.4856,135.8833,
2,Kendari,ID,74.50,moderate rain,-3.9450,122.4989,Hotel Wixel
5,Ribeira Grande,PT,70.38,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
6,Puerto Ayora,EC,73.54,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Tanabe,JP,81.61,few clouds,33.7333,135.3667,美吉屋旅館


In [82]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [83]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

In [84]:
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

In [85]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))